In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [10]:
#export 
from exp.nb_03 import *

In [11]:
x_train, y_train, x_valid, y_valid = load_data()
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)
nh, bs = 50, 64
c = y_train.max().item() + 1
loss_func = F.cross_entropy 

In [12]:
#export
class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl, self.valid_dl, self.c = train_dl, valid_dl, c

    @property
    def train_ds(self): return self.train_dl.dataset

    @property
    def valid_ds(self): return self.valid_dl.dataset

In [13]:
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [14]:
#export
def get_model(data, lr=0.5, nh=50):
    m = data.train_ds.x.shape[1]
    model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, data.c))
    return model, optim.SGD(model.parameters(), lr=lr)

class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model, self.opt, self.loss_func, self.data = model, opt, loss_func, data

In [15]:
learn = Learner(*get_model(data), loss_func, data)

In [16]:
def fit(epochs, learn):
    for e in range(epochs):
        learn.model.train()
        for xb, yb in learn.data.train_dl:
            loss = learn.loss_func(learn.model(xb), yb)
            loss.backward()
            learn.opt.step()
            learn.opt.zero_grad()

        learn.model.eval()
        with torch.no_grad():
            tot_loss, tot_acc = 0., 0.
            for xb, yb in learn.data.valid_dl:
                preds = learn.model(xb)
                tot_loss += learn.loss_func(preds, yb)
                tot_acc += accuracy(preds, yb)
        nv = len(learn.data.valid_dl)
        print(e, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv

In [17]:
loss, acc = fit(1, learn)

0 tensor(0.2674) tensor(0.9142)


## CallbackHandler

```
def one_batch(xb, yb):
    pred = model(xb)
    loss = loss_func(pred, yb)
    loss.backward()
    opt.step()
    opt.zero_grad()

def fit():
    for e in range(epochs):
        for b in train_dl: one_batch(*b)

```

## Runner

The original callback implementation in https://github.com/fastai/course-v3/blob/master/nbs/dl2/04_callbacks.ipynb uses the three functions  namely one_batch, all_batches and fit, each requiring a callback parameter in addition to a callback super-class a callback handler and the callbacks themeselves, which has been simplified by using the following Runner class

In [18]:
#export 
import re

_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')

def camel2snake(name):
    s1 = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()

class Callback():
    _order=0
    def set_runner(self, run): self.run=run
    def __getattr__(self, k): return getattr(self.run, k)
    @property
    def name(self):
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return camel2snake(name or 'callback')

In [21]:
#export

# callback in charge of switching model between training and validation modes, counting iterations and epoch progress

class TrainEvalCallback(Callback):
    def begin_fit(self):
        self.run.n_epochs=0.
        self.run.n_iter=0

    def after_batch(self):
        if not self.in_train: return
        self.run.n_epochs += 1./self.iters
        self.run.n_iter += 1
    
    def begin_epoch(self):
        self.run.n_epochs=self.epoch
        self.model.train()
        self.run.in_train=True

    def begin_validate(self):
        self.model.eval()
        self.run.in_train=False

In [22]:
class TestCallback(Callback):
    def after_step(self):
        if self.train_eval.n_iters>=10: return True

In [23]:
cbname = 'TrainEvalCallback'
camel2snake(cbname)

'train_eval_callback'

In [24]:
TrainEvalCallback().name

'train_eval'

In [7]:
#export
from typing import *

def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o] 

In [8]:
#export 

class Runner():
    def __init__(self, cbs=None, cb_funcs=None):
        cbs = listify(cbs)
        for cbf in listify(cb_funcs):
            cb = cbf()
            setattr(self, cb.name, cb)
            cbs.append(cb)
        self.stop, self.cbs = False, [TrainEvalCallback()]+cbs

    @property
    def opt(self): return self.learn.opt
    @property
    def model(self): return self.learn.model
    @property
    def loss_func(self): return self.learn.loss_func
    @property
    def data(self): return self.learn.data

    def one_batch(self, xb, yb):
        self.xb, self.yb = xb, yb
        if self('begin_batch'): return
        self.pred = self.model(self.xb)
        if self('after_pred'): return
        self.loss = self.loss_func(self.pred, self.yb)
        if self('after_loss') or not self.in_train: return
        self.loss.backward()
        if self('after_backward'): return
        self.opt.step()
        if self('after_step'): return
        self.opt.zero_grad()

    def all_batches(self, dl):
        self.iters = len(dl)
        for xb, yb in dl:
            if self.stop: break
            self.one_batch(xb, yb)
            self('after_batch')
        self.stop = False

    def fit(self, epochs, learn):
        self.epochs, self.learn = epochs, learn

        try:
            for cb in self.cbs: cb.set_runner(self)
            if self('begin_fit'): return 
            for epoch in range(epochs):
                self.epoch = epoch
                if not self('begin_epoch'): self.all_batches(self.data.train_dl)

                with torch.no_grad():
                    if not self('begin_validate'): self.all_batches(self.data.valid_dl)
                if self('after_epoch'): break


        finally:
            self('after_fit')
            self.learn = None

    def __call__(self, cb_name):
        for cb in sorted(self.cbs, key=lambda x: x._order):
            f = getattr(cb, cb_name, None)
            if f and f(): return True
        return False

In [27]:
#export

class AvgStats():
    def __init__(self, metrics, in_train): self.metrics, self.in_train = listify(metrics), in_train

    def reset(self):
        self.tot_loss, self.count = 0., 0
        self.tot_mets = [0.] * len(self.metrics)

    @property
    def all_stats(self): return [self.tot_loss.item()] + self.tot_mets
    @property
    def avg_stats(self): return [o/self.count for o in self.all_stats]

    def __repr__(self):
        if not self.count: return ""
        return f"{'train' if self.in_train else 'valid'}: {self.avg_stats}"

    def accumulate(self, run):
        bn = run.xb.shape[0]
        self.tot_loss += run.loss * bn
        self.count += bn
        for i,m in enumerate(self.metrics):
            self.tot_mets[i] += m(run.pred, run.yb) * bn


class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats, self.valid_stats = AvgStats(metrics, True), AvgStats(metrics, False)

    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()

    def after_loss(self):
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): stats.accumulate(self.run)

    def after_epoch(self):
        print(self.train_stats)
        print(self.valid_stats)

In [28]:
learn = Learner(*get_model(data), loss_func, data)

We are passing the accuracy function into the AvgStats callback, which in turn uses the AvgStats class , calling the function to calculate the specified metric by comparing batch predictions to batch actuals.

In [29]:
stats = AvgStatsCallback([accuracy])
run = Runner(cbs=stats)

In [30]:
run.fit(2, learn)

train: [0.32317119140625, tensor(0.8998)]
valid: [0.200124658203125, tensor(0.9426)]
train: [0.147077177734375, tensor(0.9540)]
valid: [0.328027978515625, tensor(0.9049)]


In [31]:
loss, acc = stats.valid_stats.avg_stats
assert acc > 0.9
loss, acc

(0.328027978515625, tensor(0.9049))

In [32]:
#export 
from functools import partial

In [33]:
acc_cbf = partial(AvgStatsCallback, accuracy)

In [34]:
run = Runner(cb_funcs=acc_cbf)

In [35]:
run.fit(1, learn)

train: [0.110900380859375, tensor(0.9662)]
valid: [0.16927784423828124, tensor(0.9506)]


In [36]:
run.avg_stats.valid_stats.avg_stats

[0.16927784423828124, tensor(0.9506)]

In [37]:
!python notebook2script.py 04_callbacks.ipynb

Converted 04_callbacks.ipynb to exp/nb_04.py


In [38]:
!python notebook2script.py 05_anneal.ipynb

Converted 05_anneal.ipynb to exp/nb_05.py
